In [1]:
%matplotlib --list

Available matplotlib backends: ['tk', 'gtk', 'gtk3', 'gtk4', 'wx', 'qt4', 'qt5', 'qt6', 'qt', 'osx', 'nbagg', 'webagg', 'notebook', 'agg', 'svg', 'pdf', 'ps', 'inline', 'ipympl', 'widget']


In [18]:
import matplotlib.pyplot as plt
import numpy as np
import math 
import functools
import matplotlib.animation as animation
import matplotlib.text as matText

from matplotlib.collections import PatchCollection
from matplotlib.patches import Arc as patchArc

#%matplotlib svg
plt.rcParams["animation.html"] = "jshtml"
#plt.rcParams["animation.html"] = "html5"
FRAMES_PER_NUMBER = 20
MAX_NUMBER_TO_CHECK = 50
plotCenter = (0,0)

numberDisplay =matText.Text(plotCenter[0], plotCenter[1], text = '2')

class CircleCycleForOnePrime:
	primeNumber = 0
	center = (0,0)
#	center = (0.5,0.5)

	logScaling = False
	radius = 0
	currentIndicatorValue = 0
	RING_WIDTH = 2
	INDICATOR_WIDTH = RING_WIDTH * 2
	ANGLE_TO_INDICATE_ZERO = 270.0
	zeroWedge = None
	nonZeroWedge = None
	indicatorWedge = None
	patches = None
	modIndicatorZero=False
	unitSlice = None
	axesForCycle = None

	def __init__(self, center, primeNumber, axes, logScaling):
		self.center = center
		self.primeNumber = int( primeNumber)
		self.setLogScaling(logScaling)
		self.axesForCycle = axes
		self.constructPrimeModCirclePatches()
	
	def setLogScaling(self, useLogScaling, minimumRadius=0):
		self.logScaling = useLogScaling
		self.radius = self.primeNumber
		if (self.logScaling):
			self.radius = math.log(self.primeNumber)
		if (self.radius < minimumRadius):
			self.radius = minimumRadius + 2
		

	
	def constructPrimeModCirclePatches(self):
		#divide a circle into primeNumber slices
		self.unitSlice = 360.0 / self.primeNumber
#		zeroRegionBegin= -(self.unitSlice/2)  
		zeroRegionBegin= 0  
		zeroRegionEnd = self.unitSlice

		self.zeroWedge=patchArc(self.center, self.radius, self.radius, theta1=zeroRegionBegin, theta2=zeroRegionEnd, linewidth=CircleCycleForOnePrime.RING_WIDTH, angle = CircleCycleForOnePrime.ANGLE_TO_INDICATE_ZERO - (self.unitSlice/2), color='red')
		self.axesForCycle.add_patch(self.zeroWedge)

		self.nonZeroWedge=patchArc(self.center, self.radius, self.radius, theta1=zeroRegionEnd, theta2=zeroRegionBegin,linewidth=CircleCycleForOnePrime.RING_WIDTH,angle = CircleCycleForOnePrime.ANGLE_TO_INDICATE_ZERO - (self.unitSlice/2), color='green')
		self.axesForCycle.add_patch(self.nonZeroWedge)
		
		self.indicatorWedge = patchArc(self.center, self.radius, self.radius, theta1= -self.unitSlice / 5, theta2= self.unitSlice/5, linewidth=CircleCycleForOnePrime.INDICATOR_WIDTH, angle = CircleCycleForOnePrime.ANGLE_TO_INDICATE_ZERO - (self.unitSlice/2), color='blue')
		self.axesForCycle.add_patch(self.indicatorWedge)
		patches=[self.zeroWedge, self.nonZeroWedge, self.indicatorWedge]
		self.currentIndicatorValue = 0
												

	def setIndicator(self, newIndicatorValue):
		#convert the remainder to a percent
		self.currentIndicatorValue = (newIndicatorValue % self.primeNumber)
		self.modIndicatorZero = (self.currentIndicatorValue == 0)
#		print("mod indic for " , newIndicatorValue, " currentIndic " , self.currentIndicatorValue, " zero indic ", self.modIndicatorZero)
		translatedFromValueToAngle = CircleCycleForOnePrime.ANGLE_TO_INDICATE_ZERO - (self.unitSlice/2) + self.unitSlice*self.currentIndicatorValue
		self.indicatorWedge.set_angle(translatedFromValueToAngle)

	def getPatches(self):
		return self.patches


largestPrimeFound = 2


# Fixing random state for reproducibility
np.random.seed(19680801)


fig = plt.figure()
ax = fig.subplots()

#patches = []
cycles = []
twoCycle = CircleCycleForOnePrime(plotCenter, 2, ax, False)

#numberDisplay =matText.Text(plotCenter[0], plotCenter[1], text = '2')
ax.add_artist(numberDisplay)
cycles.append(twoCycle)
ax.autoscale()
startingValue=2
#ax.add_collection(oneCycle.getPatchCollection())
#The main driving loop

def animate(frameNumber, cycleContainer):
	currentNumber = startingValue + frameNumber / FRAMES_PER_NUMBER
	#only checxk for a new cycle if this is an integer
	currentNumberIsIntegerAndNotDivisibleByAnyCycle = (currentNumber == math.floor(currentNumber))

	
	numberDisplay.set(text=int(currentNumber))

#	print('animate checknumber ',currentNumber,' ',  currentNumberIsIntegerAndNotDivisibleByAnyCycle)
	patchList = []
	for oneCycle in cycleContainer:
		oneCycle.setIndicator(currentNumber)
		patchList.append(oneCycle.getPatches())
		#check if current number is divisible by cycle prime
		if (currentNumberIsIntegerAndNotDivisibleByAnyCycle):
			currentNumberIsIntegerAndNotDivisibleByAnyCycle = currentNumberIsIntegerAndNotDivisibleByAnyCycle and not oneCycle.modIndicatorZero
#			print('in cycle for ' + str(oneCycle.primeNumber) +' checking ' + str(currentNumber) + ' value ' +   str(currentNumberIsIntegerAndNotDivisibleByAnyCycle))
		
	#if current number isn't divisible by any cycle, it's a prime
	if (currentNumberIsIntegerAndNotDivisibleByAnyCycle):
		#got a new prime
		newCycle = CircleCycleForOnePrime(plotCenter, currentNumber, ax, False)
		cycles.append(newCycle)
#		patchList.append(newCycle.getPatches())
		patchList.append(newCycle.indicatorWedge )
#		ax.add_collection(newCycle.getPatchCollection())
		ax.autoscale_view()
	return patchList

anim = functools.partial(animate, cycleContainer=cycles)
numberOfFrames = FRAMES_PER_NUMBER * MAX_NUMBER_TO_CHECK
ani = animation.FuncAnimation(fig, anim, numberOfFrames, save_count= 100, repeat=False, blit=True)
ani.save(filename='primeClock.html', writer='html')
plt.show()
#put this here so ani doesn't get garbage collected
ani



<IPython.core.display.Javascript object>

/tmp/ipykernel_1893/2854199702.py:140: UserWarning: You passed in an explicit save_count=100 which is being ignored in favor of frames=1000.
  ani = animation.FuncAnimation(fig, anim, numberOfFrames, save_count= 100, repeat=False, blit=True)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math 
import functools
import matplotlib.animation as animation
import matplotlib.text as matText

from matplotlib.collections import PatchCollection
from matplotlib.patches import Arc as patchArc

%matplotlib notebook
plt.rcParams["animation.html"] = "jshtml"

RING_WIDTH = 5
INDICATOR_WIDTH = RING_WIDTH * 2
ANGLE_TO_INDICATE_ZERO = 270.0

fig, ax = plt.subplots()

primeNumber = 2

unitSlice = 360.0 / primeNumber
#the zero region should  be at the bottom, so 270 +/- 1/2 unit slice
zeroRegionBegin=- (unitSlice/2)  
zeroRegionEnd = (unitSlice/2)

center = (0.5,0.5)
radius = .4
zeroWedge=patchArc(center, radius,radius, theta1=zeroRegionBegin, theta2=zeroRegionEnd, linewidth=RING_WIDTH, angle = ANGLE_TO_INDICATE_ZERO, color='red')
ax.add_patch(zeroWedge)

nonZeroWedge=patchArc(center, radius, radius, theta1=zeroRegionEnd, theta2=zeroRegionBegin,linewidth=RING_WIDTH,angle = ANGLE_TO_INDICATE_ZERO, color='green')
ax.add_patch(nonZeroWedge)
indicatorWedge = patchArc(center, radius, radius, theta1= -unitSlice / 5, theta2= unitSlice/5, linewidth=INDICATOR_WIDTH, angle = ANGLE_TO_INDICATE_ZERO, color='blue')
ax.add_patch(indicatorWedge)

currentIndicatorValue = 0
cycleCollection = [zeroWedge, nonZeroWedge, indicatorWedge]
#p = PatchCollection(cycleCollection, match_original=False)
#p.set_array(colors)
#ax.add_collection(p)

numberDisplay =matText.Text(center[0], center[1], text = '1')
ax.add_artist(numberDisplay)


def animate(frameNumber,oneArc, cycleContainer):
	# Simulate new data coming in.
	currentNumber = frameNumber / 5
	modNumber = math.floor(currentNumber)
#	numberDisplay.set(text=modNumber)
	numberDisplay.set(text=currentNumber)
	arcAngle = (ANGLE_TO_INDICATE_ZERO + ((currentNumber %7)/7.0)  * 360) % 360
	oneArc.set_angle( arcAngle)
	return cycleContainer

anim = functools.partial(animate, oneArc=indicatorWedge, cycleContainer=cycleCollection)
ani = animation.FuncAnimation(fig, anim, 50, repeat=False, blit=True)
plt.show()
ani

